In [1]:
# Import packages
import numpy as np
import pandas as pd

In [2]:
# Load soil temperature data

df = pd.read_csv('fargo_hourly_deep_soil_temperature.csv')

FileNotFoundError: File b'fargo_hourly_deep_soil_temperature.csv' does not exist

In [8]:
df.head(5)

,station,time_cst,T5cm,T10cm,T20cm,T30cm,T40cm,T50cm,T60cm,T80cm,T100cm,T125cm,T150cm,T175cm,T200cm,T225cm
0,Fargo,10/2/14 12:00,18.09,15.36,14.49,14.64,14.71,14.68,14.60,14.28,13.96,13.43,12.92,12.37,11.66,11.16
1,Fargo,10/2/14 13:00,18.96,16.08,14.70,14.66,14.72,14.68,14.60,14.25,13.92,13.38,12.86,12.30,11.66,11.15
2,Fargo,10/2/14 14:00,18.59,16.45,14.89,14.70,14.72,14.65,14.56,14.20,13.85,13.30,12.78,12.22,11.65,11.10
3,Fargo,10/2/14 15:00,19.82,17.01,15.10,14.73,14.71,14.65,14.54,14.18,13.80,13.26,12.71,12.12,11.65,11.13
4,Fargo,10/2/14 16:00,19.09,17.14,15.24,14.74,14.68,14.59,14.49,14.10,13.73,13.14,12.60,11.97,11.64,11.08


In [45]:
df['date'] = pd.to_datetime(df.time_cst)

In [47]:
df['DOY'] = df.date.dt.dayofyear
df.head(5)

,station,time_cst,T5cm,T10cm,T20cm,T30cm,T40cm,T50cm,T60cm,T80cm,T100cm,T125cm,T150cm,T175cm,T200cm,T225cm,date,DOY
0,Fargo,10/2/14 12:00,18.09,15.36,14.49,14.64,14.71,14.68,14.60,14.28,13.96,13.43,12.92,12.37,11.66,11.16,2014-10-02 12:00:00,275
1,Fargo,10/2/14 13:00,18.96,16.08,14.70,14.66,14.72,14.68,14.60,14.25,13.92,13.38,12.86,12.30,11.66,11.15,2014-10-02 13:00:00,275
2,Fargo,10/2/14 14:00,18.59,16.45,14.89,14.70,14.72,14.65,14.56,14.20,13.85,13.30,12.78,12.22,11.65,11.10,2014-10-02 14:00:00,275
3,Fargo,10/2/14 15:00,19.82,17.01,15.10,14.73,14.71,14.65,14.54,14.18,13.80,13.26,12.71,12.12,11.65,11.13,2014-10-02 15:00:00,275
4,Fargo,10/2/14 16:00,19.09,17.14,15.24,14.74,14.68,14.59,14.49,14.10,13.73,13.14,12.60,11.97,11.64,11.08,2014-10-02 16:00:00,275


In [134]:
T5cm_avg = round(df.T5cm.groupby(df.DOY).mean(),1)
T50cm_avg = round(df.T50cm.groupby(df.DOY).mean(),1)


In [39]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper
from bokeh.palettes import Viridis256 # 3 to 11, 256
from bokeh.transform import transform, linear_cmap
from bokeh.models import ColumnDataSource, ColorBar # ColumnDataSource is the link between Bokeh and Pandas
output_notebook()

Loading BokehJS ...

In [88]:
# Soil temperature bar (single temperature depth)
p = figure(plot_width=800, plot_height=200)

xdata = list(range(1,367))
ydata = np.ones(len(xdata)) * 5;
zdata = T5cm_avg

mapper = linear_cmap(field_name='z', palette=Viridis256 ,low=min(zdata) ,high=max(zdata))
source = ColumnDataSource(dict(x=xdata, y=ydata, z=zdata))
p.square(x='x', y='y', line_color=mapper, color=mapper, fill_alpha=1, size=50, source=source)
color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))
p.add_layout(color_bar, 'right')

show(p)

# If you zoom, you will see that the squares do not touch and there is white space. If we want to
# prevent this and let the markers fill the white space, we need to use a lower level glyph

# square is a high-level glyph, that only requires few properties. To handle a different sizes 
# we need to use the low-level Rect glyph, which allow us to control more details.

# For instance, you may think why don't we just set the 'width' of the square to a different value,
# so that the markers can fill the space between neighboring markers. If we add this property (and I
# encourage you to do so), we obtain the following error message:

# p.square(x='x', y='y', width=1, line_color=mapper, color=mapper, fill_alpha=1, size=50, source=source)
# AttributeError: unexpected attribute 'width' to Square, similar attributes are line_width

# In other, words, the square glyph does not have such property. The size of the square is thorugh the
# 'size' attribute.

# High-level means: ready out of the box, fewer choices, quicker
# Low-level means: more options to control, slower, ideal to achieve a specific effect


In [78]:
#xdata = list(range(1,367))
[np.ones(5) * 5, np.ones(5) * 5]

[array([5., 5., 5., 5., 5.]), array([5., 5., 5., 5., 5.])]

In [93]:
# Soil temperature bar (single temperature depth)
from bokeh.models.glyphs import Rect

p = figure(plot_width=800, plot_height=200)

xdata = list(range(1,367))
ydata = np.ones(len(xdata)) * 5;
zdata = T5cm_avg

mapper = linear_cmap(field_name='z', palette=Viridis256 ,low=min(zdata) ,high=max(zdata))
source = ColumnDataSource(dict(x=xdata, y=ydata, z=zdata))

# High level 'Rect' glyph
glyph = Rect(x='x', y='y', width=1, height=10, angle=0, line_color=mapper, fill_color=mapper, fill_alpha=1)
p.add_glyph(source, glyph)

color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))
p.add_layout(color_bar, 'right')

show(p)

In [135]:
# Soil temperature bar (single temperature depth)
from bokeh.models.glyphs import Rect
from bokeh.models import HoverTool

TOOLTIPS = [
    ('DOY', '$index'),
    ('5 cm', '@T5cm_avg Celsius'),
    ('50 cm', '@T50cm_avg Celsius')
]

p = figure(plot_width=800, plot_height=200, tooltips=TOOLTIPS, title='Average soil temperature in Fargo, ND')

xdata = list(range(1,367))
ydata_5cm = np.ones(len(xdata)) * -5; 
ydata_50cm = np.ones(len(xdata)) * -50;
# Negative values, so that we mimic the soil profile

# We need a mapper for each. We will keep the min and max values of the surface, so that the scale
# for linear interpolation remains constant.
mapper_5cm = linear_cmap(field_name='T5cm_avg', palette=Viridis256 ,low=min(zdata) ,high=max(zdata))
mapper_50cm = linear_cmap(field_name='T50cm_avg', palette=Viridis256 ,low=min(zdata) ,high=max(zdata))

source = ColumnDataSource(dict(x=xdata, y5=ydata_5cm, y50=ydata_50cm, T5cm_avg=T5cm_avg, T50cm_avg=T50cm_avg))

# A height of 10 will assume that the temperature is representative of 5 cm above and below the sensor
# Temperature at 5 cm
glyph_5cm = Rect(x='x', y='y5', width=1, height=10, angle=0, line_color=mapper_5cm, fill_color=mapper_5cm, fill_alpha=1)
p.add_glyph(source, glyph_5cm)

# Temperatue at 50 cm
glyph_50cm = Rect(x='x', y='y50', width=1, height=10, angle=0, line_color=mapper_50cm, fill_color=mapper_50cm, fill_alpha=1)
p.add_glyph(source, glyph_50cm)

# Create and add colorbar
color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))
p.add_layout(color_bar, 'right')


show(p)